In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import armagarch as ag
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats.distributions import chi2 
from arch import arch_model
from math import sqrt
import seaborn as sns
sns.set()


In [36]:
sp_futures = pd.read_csv("S&P500FuturesHistoricalData.csv")
df_full = sp_futures[['Date', 'Price', 'Change %']]
size = int(len(df_full)*0.8)
df, df_test = df_full.iloc[:size], df_full.iloc[size:]
print(len(df), len(df_test))

2112 528


In [37]:
df['Price'] = [float(x.replace(',', '')) for x in df['Price']]
df['Change %'] = [float(x.strip('%')) for x in df['Change %']]
df.Date = pd.to_datetime(df.Date, dayfirst = True)
df.set_index('Date', inplace=True)
df.columns = [['market_value', 'returns']]
df

/Users/ischaider/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/ischaider/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/ischaider/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,market_value,returns
Date,,
2022-11-11,4000.25,0.99
2022-10-11,3961.00,5.47
2022-09-11,3755.50,-2.08
2022-08-11,3835.25,0.52
2022-07-11,3815.25,0.95
...,...,...
2014-08-12,2059.50,-0.79
2014-05-12,2076.00,0.19
2014-04-12,2072.00,-0.02


In [38]:
def LLR_test(mod_1, mod_2, DF = 1):
    L1 = mod_1.fit(start_ar_lags = 11).llf
    L2 = mod_2.fit(start_ar_lags = 11).llf
    LR = (2*(L2-L1))    
    p = chi2.sf(LR, DF).round(3)
    return p

In [39]:
model_garch_1_1 = arch_model(df.returns[1:], mean = "Constant", vol = "GARCH", p = 1, q = 1)
results_garch_1_1 = model_garch_1_1.fit(update_freq = 5)
results_garch_1_1.summary()


Iteration:      5,   Func. Count:     35,   Neg. LLF: 2642.865887035423
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2641.8264221855643
            Iterations: 9
            Function evaluations: 55
            Gradient evaluations: 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:           ('returns',)   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2641.83
Distribution:                  Normal   AIC:                           5291.65
Method:            Maximum Likelihood   BIC:                           5314.27
                                        No. Observations:                 2111
Date:                Sun, Nov 13 2022   Df Residuals:                     2110
Time:                        21:45:23   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0784  1.368e-02      5.730  1.003e-08 [5.156e-02,  0.105]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0228  5.979e-03      3.816  1.354e-04 [1.110e-02,3.454e-02]
alpha[1]       0.2322  2.904e-02      7.995  1.291e-15     [  0.175,  0.289]
beta[1]        0.7678  2.619e-02     29.315 6.616e-189     [  0.716,  0.819]
============================================================================

Covariance estimator: robust
"""

In [40]:
model_garch_1_2 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 1, q = 2)
results_garch_1_2 = model_garch_1_2.fit(update_freq = 5)
results_garch_1_2.summary()

Iteration:      5,   Func. Count:     41,   Neg. LLF: 2648.6897660611808
Iteration:     10,   Func. Count:     71,   Neg. LLF: 2641.8264392580813
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2641.8264221535164
            Iterations: 12
            Function evaluations: 82
            Gradient evaluations: 12


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:           ('returns',)   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2641.83
Distribution:                  Normal   AIC:                           5293.65
Method:            Maximum Likelihood   BIC:                           5321.93
                                        No. Observations:                 2111
Date:                Sun, Nov 13 2022   Df Residuals:                     2110
Time:                        21:45:41   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0784  1.357e-02      5.773  7.776e-09 [5.176e-02,  0.105]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0228  5.684e-03      4.015  5.954e-05 [1.168e-02,3.396e-02]
alpha[1]       0.2322  2.437e-02      9.527  1.620e-21     [  0.184,  0.280]
beta[1]        0.7678      0.174      4.425  9.652e-06     [  0.428,  1.108]
beta[2]        0.0000      0.164      0.000      1.000     [ -0.322,  0.322]
============================================================================

Covariance estimator: robust
"""

In [41]:
model_garch_1_3 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 1, q = 3)
results_garch_1_3 = model_garch_1_3.fit(update_freq = 5)
results_garch_1_3.summary()

Iteration:      5,   Func. Count:     45,   Neg. LLF: 2655.008436156376
Iteration:     10,   Func. Count:     80,   Neg. LLF: 2641.8264367097768
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2641.826422275934
            Iterations: 12
            Function evaluations: 93
            Gradient evaluations: 12


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:           ('returns',)   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2641.83
Distribution:                  Normal   AIC:                           5295.65
Method:            Maximum Likelihood   BIC:                           5329.58
                                        No. Observations:                 2111
Date:                Sun, Nov 13 2022   Df Residuals:                     2110
Time:                        21:45:49   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0784  1.357e-02      5.772  7.817e-09 [5.175e-02,  0.105]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0228  5.682e-03      4.016  5.919e-05 [1.168e-02,3.396e-02]
alpha[1]       0.2322  2.576e-02      9.014  1.985e-19     [  0.182,  0.283]
beta[1]        0.7678      0.171      4.486  7.256e-06     [  0.432,  1.103]
beta[2]    2.2727e-11      0.243  9.367e-11      1.000     [ -0.476,  0.476]
beta[3]    1.1566e-11      0.145  7.971e-11      1.000     [ -0.284,  0.284]
============================================================================

Covariance estimator: robust
"""

In [42]:
model_garch_2_1 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 2, q = 1)
results_garch_2_1 = model_garch_2_1.fit(update_freq = 5)
results_garch_2_1.summary()

Iteration:      5,   Func. Count:     40,   Neg. LLF: 2933.060140530726
Iteration:     10,   Func. Count:     73,   Neg. LLF: 2630.8918390924055
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2630.8918390924055
            Iterations: 11
            Function evaluations: 78
            Gradient evaluations: 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:           ('returns',)   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2630.89
Distribution:                  Normal   AIC:                           5271.78
Method:            Maximum Likelihood   BIC:                           5300.06
                                        No. Observations:                 2111
Date:                Sun, Nov 13 2022   Df Residuals:                     2110
Time:                        21:46:00   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0800  1.345e-02      5.949  2.699e-09 [5.366e-02,  0.106]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0353  9.986e-03      3.536  4.057e-04 [1.574e-02,5.489e-02]
alpha[1]       0.1187  2.994e-02      3.964  7.383e-05   [6.000e-02,  0.177]
alpha[2]       0.2005  5.580e-02      3.592  3.276e-04   [9.109e-02,  0.310]
beta[1]        0.6809  5.015e-02     13.575  5.602e-42     [  0.583,  0.779]
============================================================================

Covariance estimator: robust
"""

In [43]:
model_garch_3_1 = arch_model(df.returns[1:], mean = "Constant",  vol = "GARCH", p = 3, q = 1)
results_garch_3_1 = model_garch_3_1.fit(update_freq = 5)
results_garch_3_1.summary()

Iteration:      5,   Func. Count:     46,   Neg. LLF: 2652.4106616888093
Iteration:     10,   Func. Count:     85,   Neg. LLF: 2630.891924825736
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2630.8918390778463
            Iterations: 13
            Function evaluations: 105
            Gradient evaluations: 13


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:           ('returns',)   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2630.89
Distribution:                  Normal   AIC:                           5273.78
Method:            Maximum Likelihood   BIC:                           5307.71
                                        No. Observations:                 2111
Date:                Sun, Nov 13 2022   Df Residuals:                     2110
Time:                        21:46:07   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0800  1.349e-02      5.930  3.021e-09 [5.358e-02,  0.106]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0353  1.223e-02      2.888  3.883e-03 [1.134e-02,5.928e-02]
alpha[1]       0.1187  2.984e-02      3.977  6.978e-05   [6.020e-02,  0.177]
alpha[2]       0.2005  5.772e-02      3.473  5.153e-04   [8.732e-02,  0.314]
alpha[3]   2.7032e-12  7.248e-02  3.730e-11      1.000     [ -0.142,  0.142]
beta[1]        0.6809  7.340e-02      9.276  1.762e-20     [  0.537,  0.825]
============================================================================

Covariance estimator: robust
"""

In [44]:
# load data from KennethFrench library
# ff = web.DataReader('F-F_Research_Data_Factors_daily', 'famafrench')
# ff = ff[0]

# define mean, vol and distribution
# meanMdl = ag.ARMA(order = {'AR':1,'MA':0})
# volMdl = ag.garch(order = {'p':1,'q':1})
# distMdl = ag.normalDist()

# create a model
# model = ag.empModel(sp_futures['Price'].to_frame(), meanMdl, volMdl, distMdl)
# model = ag.empModel(ff['Mkt-RF'].to_frame(), meanMdl, volMdl, distMdl)

# fit model
# model.fit()

# get the conditional mean
# Ey = model.Ey

# get conditional variance
# ht = model.ht
# cvol = np.sqrt(ht)

# get standardized residuals
# stres = model.stres

# make a prediction of mean and variance over next 3 days.
# pred = model.predict(nsteps = 3)

# pred